<a href="https://colab.research.google.com/github/shruthibalaji2307/-NotMe---Chrome-extension/blob/master/LSTM_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk

In [2]:
!pip install keras

In [3]:
!pip install chart-studio
!pip install cufflinks

     |████████████████████████████████| 81kB 32.2MB/s 
     |████████████████████████████████| 81kB 31.1MB/s 
     |████████████████████████████████| 41.5MB 93.5MB/s 
  Created wheel for cufflinks: filename=cufflinks-0.16-cp36-none-any.whl size=67603 sha256=f7ae98d4730c4785744a603806586a1f1e1664c0e15c48480166d3ff90040e2f
  Stored in directory: /root/.cache/pip/wheels/8d/5a/6f/c97d47dc901071611809eb61aaa477d50a60692dc764dca622
Successfully built cufflinks
  Found existing installation: plotly 4.1.1
    Uninstalling plotly-4.1.1:
      Successfully uninstalled plotly-4.1.1


In [4]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import chart_studio.plotly as py
import plotly.graph_objs as go
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
from google.colab import files
uploaded = files.upload()

Saving labeled_data1.csv to labeled_data1.csv


In [0]:
df = pd.read_csv('labeled_data1.csv')

In [7]:
df.head()

,column_a,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,None,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,Offensive,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,Offensive,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,Offensive,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,Offensive,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [0]:
df.drop(['column_a','count','hate_speech','offensive_language','neither'],axis=1,inplace=True)

In [9]:
df.head()

,class,tweet
0,None,!!! RT @mayasolovely: As a woman you shouldn't...
1,Offensive,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,Offensive,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,Offensive,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,Offensive,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 2 columns):
class    24783 non-null object
tweet    24783 non-null object
dtypes: object(2)
memory usage: 387.3+ KB


In [0]:
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['tweet'] = df['tweet'].apply(clean_text)

In [12]:
df.head()

,class,tweet
0,None,rt mayasolovely woman shouldnt complain cleani...
1,Offensive,rt mleew17 boy dats coldtyga dwn bad cuffin da...
2,Offensive,rt urkindofbrand dawg rt 80sbaby4life ever fuc...
3,Offensive,rt c_g_anderson viva_based look like tranny
4,Offensive,rt shenikaroberts shit hear might true might f...


In [13]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['tweet'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 37512 unique tokens.


In [14]:
X = tokenizer.texts_to_sequences(df['tweet'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (24783, 250)


In [15]:
Y = pd.get_dummies(df['class']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (24783, 3)


In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(22304, 250) (22304, 3)
(2479, 250) (2479, 3)


In [17]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 100)          5000000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 250, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 303       
Total params: 5,080,703
Trainable params: 5,080,703
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Train on 20073 samples, validate on 2231 samples
Epoch 1/5
20073/20073 [==============================] - 147s 7ms/step - loss: 0.0271 - acc: 0.9913 - val_loss: 0.5012 - val_acc: 0.8879
Epoch 2/5
20073/20073 [==============================] - 147s 7ms/step - loss: 0.0186 - acc: 0.9948 - val_loss: 0.5392 - val_acc: 0.8857
Epoch 3/5
20073/20073 [==============================] - 147s 7ms/step - loss: 0.0153 - acc: 0.9954 - val_loss: 0.5993 - val_acc: 0.8749
Epoch 4/5
20073/20073 [==============================] - 146s 7ms/step - loss: 0.0133 - acc: 0.9957 - val_loss: 0.6641 - val_acc: 0.8821


In [21]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

2479/2479 [==============================] - 12s 5ms/step
Test set
  Loss: 0.774
  Accuracy: 0.864
